In [11]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Fungsi untuk menjalankan query SQL
def execute_query(query):
    try:
        # Menghubungkan ke server MySQL atau MariaDB
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="QTI-Employee"
        )

        if connection.is_connected():
            # Membuat objek cursor untuk eksekusi query
            cursor = connection.cursor()

            # Menjalankan query
            cursor.execute(query)

            # Mengambil hasil query (jika ada)
            result = cursor.fetchall()

            # Menutup koneksi (tanpa menutup kursor)
            connection.close()

            return result, cursor.description

    except Error as e:
        print(f"Error: {e}")

# Contoh penggunaan
query_employees = "SELECT * FROM employees"
result_employees, description_employees = execute_query(query_employees)

query_assets = "SELECT * FROM assets"
result_assets, description_assets = execute_query(query_assets)

query_losses = "SELECT * FROM losses"
result_losses, description_losses = execute_query(query_losses)

# Menampilkan hasil query dalam bentuk DataFrame
df_employees = pd.DataFrame(result_employees, columns=[i[0] for i in description_employees])
df_assets = pd.DataFrame(result_assets, columns=[i[0] for i in description_assets])
df_losses = pd.DataFrame(result_losses, columns=[i[0] for i in description_losses])

# Menampilkan DataFrame
print("Data Tabel Employees:")
print(df_employees)

print("\nData Tabel Assets:")
print(df_assets)

print("\nData Tabel Losses:")
print(df_losses)


Data Tabel Employees:
   employee_id employee_name company_name employee_number   position  \
0            1      John Doe            X            E123    Manager   
1            2    Jane Smith            X            E456  Developer   
2            3   Bob Johnson            X            E789    Analyst   

  department     division  
0         HR   Operations  
1         IT  Development  
2    Finance    Analytics  

Data Tabel Assets:
   asset_id asset_name  asset_type  brand asset_number  employee_id
0       101     Laptop  Electronic     HP         L101            1
1       102    Desktop  Electronic   Dell         D102            2
2       103    Printer  Peripheral  Epson         P103            3

Data Tabel Losses:
   loss_id                 time     city    province  asset_id
0     1001  2023-01-01 10:00:00  Bandung  Jawa Barat       101
1     1002  2023-02-15 15:30:00   Bekasi  Jawa Barat       102
2     1003  2023-03-20 09:45:00    Depok  Jawa Barat       103


In [10]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Fungsi untuk menjalankan query SQL
def execute_query(query):
    try:
        # Menghubungkan ke server MySQL atau MariaDB
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="QTI-Employee"
        )

        if connection.is_connected():
            # Membuat objek cursor untuk eksekusi query
            cursor = connection.cursor()

            # Menjalankan query
            cursor.execute(query)

            # Mengambil hasil query (jika ada)
            result = cursor.fetchall()

            if cursor.description:
                description = cursor.description
            else:
                description = []

            # Menutup koneksi (tanpa menutup kursor)
            connection.close()

            return result, description

    except Error as e:
        print(f"Error: {e}")

        # Mengembalikan None jika terjadi kesalahan
        return None, None

if __name__ == "__main__":
    # Query 1: Jenis barang apa saja yang sering hilang di tiap kota dalam provinsi Jawa Barat
    query_1 = """
    SELECT a.asset_name, a.asset_type, l.province, COUNT(l.loss_id) AS total_losses
    FROM assets a
    JOIN losses l ON a.asset_id = l.asset_id
    WHERE l.province LIKE 'Jawa Barat%'
    GROUP BY a.asset_name, a.asset_type, l.province
    ORDER BY l.province, total_losses DESC;
    """

    # Query 2: 1 atasan di setiap perusahaan dimana para staff yang dia miliki sering menghilangkan asset
    query_2 = """
    SELECT e.employee_name AS boss_name, e.position AS boss_position,
           e.department AS boss_department, e.division AS boss_division,
           COUNT(l.loss_id) AS total_losses
    FROM employees e
    JOIN assets a ON e.employee_id = a.employee_id
    JOIN losses l ON a.asset_id = l.asset_id
    GROUP BY e.employee_id
    ORDER BY total_losses DESC
    LIMIT 1;
    """

    # Menampilkan hasil query 1 dalam bentuk DataFrame
    result_query_1, description_query_1 = execute_query(query_1)
    df_query_1 = pd.DataFrame(result_query_1, columns=[i[0] for i in description_query_1])
    print("Query 1:")
    print(df_query_1)

    # Menampilkan hasil query 2 dalam bentuk DataFrame
    result_query_2, description_query_2 = execute_query(query_2)
    df_query_2 = pd.DataFrame(result_query_2, columns=[i[0] for i in description_query_2])
    print("\nQuery 2:")
    print(df_query_2)


Query 1:
  asset_name  asset_type    province  total_losses
0     Laptop  Electronic  Jawa Barat             1
1    Desktop  Electronic  Jawa Barat             1
2    Printer  Peripheral  Jawa Barat             1

Query 2:
  boss_name boss_position boss_department boss_division  total_losses
0  John Doe       Manager              HR    Operations             1
